In [112]:
import numpy as np
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
import json
import os
from itertools import product

In [113]:
url = 'http://185.63.190.188/trade82new/hs/Service/AIExchange/ВремяПоставкиТоваров'

In [135]:
# Функция считающая процентиль для агрегации
def percentile(n):
    def percentile_(sroki):
        return np.percentile(sroki, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [115]:
def object_to_date(deliv_time, deliv_time_features, name_features):
    deliv_time[name_features] = pd.to_datetime(
        deliv_time_features,
        format="%Y-%m-%dT%H:%M:%S"
    )

In [116]:
headers = {
    'Content-type': 'application/json',
}
data = {
    "НачалоПериода": "10.01.2021 0:00:00",
    "КонецПериода": "12.01.2021 23:59:59",
    "ВидПоставкиТоваров": "Со свободных остатков"
}

In [117]:
def load_data_from_api(service_url):
    response = requests.post(
        service_url,
        auth=HTTPBasicAuth('Web', 'WebMarket'),
        data=json.dumps(data),
        headers=headers
    )
    return response.json()

In [118]:
list_to_drop = [
    'item_id', 'fabricator_id', 'item_id',
    'item_guid', 'item_charac_id', 'item_charac_guid'
]

## 1) Разбиение Мск-Екб и Мск-Чел по процентилям

In [220]:
# Посмотрим локальную директорию
os.listdir('C:/Users/2/PycharmProjects/delivery_time_statistics/datasets')

['delivery_time.csv', 'graphics.csv']

In [221]:
# Загрузим датасет
deliv = pd.read_csv('/Users/2/PycharmProjects/delivery_time_statistics/datasets/delivery_time.csv', dtype='unicode')

In [222]:
deliv.head(5)

,date_receipt,warehouse_id,warehouse_guid,item_id,item_guid,item_charac_id,item_charac_guid,date_orders,fabricator_id,fabricator_guid,IsFromSupplier
0,2020-10-21T00:00:00,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000129111,ad818131-ee77-11e9-a2dc-000c29675a27,008,a2f3bc1e-d22a-41f3-8ab6-0942d1cc411b,2020-09-23T00:00:00,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0
1,2020-12-07T00:00:00,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000056959,07e33f70-5d4a-11e7-ab48-000c2945f18b,001,07e33f72-5d4a-11e7-ab48-000c2945f18b,2020-12-02T00:00:00,ÐÐ0000015,f6548095-c106-11e0-8baa-00262d126e8b,0
2,2020-10-19T00:00:00,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000056334,02701d8b-55ab-11e7-ab48-000c2945f18b,001,02701d8d-55ab-11e7-ab48-000c2945f18b,2020-09-20T00:00:00,ÐÐ0000015,f6548095-c106-11e0-8baa-00262d126e8b,0
3,2020-12-03T00:00:00,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000129187,4ddc3245-ef0e-11e9-a2dc-000c29675a27,003,4a169e5d-725b-4f5b-a85b-d3a6dd88732d,2020-11-29T00:00:00,ÐÐ0000015,f6548095-c106-11e0-8baa-00262d126e8b,0
4,2020-10-30T00:00:00,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000135901,98d2b397-10f6-11ea-a2e9-00505607eae9,002,5faa48df-95ba-4493-9e23-f8c95ad4b55b,2020-06-17T00:00:00,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0


In [188]:
# Отфильтруем перемещения по Москва-Екатеринбург
msk_ekb = deliv.loc[
    (deliv['fabricator_guid'] == 'a2f89531-75de-11e8-a2c3-000c29dcdf3d') \
    & (deliv['warehouse_guid'] == '5f219248-fadd-11df-8a53-00262d126e8b')
]

In [223]:
# Отфильтруем перемещения по Москва-Челябинск
msk_chel = deliv.loc[
    (deliv['fabricator_guid'] == 'a2f89531-75de-11e8-a2c3-000c29dcdf3d') \
    & (deliv['warehouse_guid'] == 'f6548095-c106-11e0-8baa-00262d126e8b')
]

In [224]:
msk_chel.sample(5)

,date_receipt,warehouse_id,warehouse_guid,item_id,item_guid,item_charac_id,item_charac_guid,date_orders,fabricator_id,fabricator_guid,IsFromSupplier
22865,2020-12-23T00:00:00,0000015,f6548095-c106-11e0-8baa-00262d126e8b,000044882,8be534aa-a0c6-11e6-80d2-00155d000e07,028,5d57d07a-4e02-11e8-a2c1-000c29dcdf3d,2020-12-14T00:00:00,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0
29475,2020-12-23T00:00:00,0000015,f6548095-c106-11e0-8baa-00262d126e8b,000053951,c6a906a1-2f1b-11e7-80d9-00155d000e07,001,c6a906a3-2f1b-11e7-80d9-00155d000e07,2020-12-12T00:00:00,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0
21123,2020-10-30T00:00:00,0000015,f6548095-c106-11e0-8baa-00262d126e8b,000126565,48a52deb-df67-11e9-a2d9-000c29675a27,001,7ac45180-df67-11e9-a2d9-000c29675a27,2020-09-12T00:00:00,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0
22037,2020-10-28T00:00:00,0000015,f6548095-c106-11e0-8baa-00262d126e8b,000157238,6d8aa53c-7a34-11ea-a307-00505607eae9,0000000001,11281a2b-7a35-11ea-a307-00505607eae9,2020-10-03T00:00:00,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0
31946,2020-12-23T00:00:00,0000015,f6548095-c106-11e0-8baa-00262d126e8b,000149876,3deb1c7e-54e6-11ea-a307-00505607eae9,0000000001,3deb1c80-54e6-11ea-a307-00505607eae9,2020-12-13T00:00:00,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0


In [225]:
# Сделаем тип дата
object_to_date(msk_chel, msk_chel.date_receipt, 'date_receipt')
object_to_date(msk_chel, msk_chel.date_orders, 'date_orders')

C:\Users\2\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [226]:
msk_chel['difference'] = msk_chel.date_receipt - msk_chel.date_orders
msk_chel.sample(5)

C:\Users\2\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,date_receipt,warehouse_id,warehouse_guid,item_id,item_guid,item_charac_id,item_charac_guid,date_orders,fabricator_id,fabricator_guid,IsFromSupplier,difference
22084,2020-10-21,0000015,f6548095-c106-11e0-8baa-00262d126e8b,000103761,808c68a4-359e-11e9-a2ce-000c29675a27,001,808c68a6-359e-11e9-a2ce-000c29675a27,2020-10-12,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,9 days
33606,2021-01-16,0000015,f6548095-c106-11e0-8baa-00262d126e8b,000136705,b78ef70b-14e9-11ea-a2ec-00505607eae9,003,1e8ea5a3-14ec-11ea-a2ec-00505607eae9,2021-01-04,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,12 days
33506,2021-01-09,0000015,f6548095-c106-11e0-8baa-00262d126e8b,000129058,ba45b331-ee62-11e9-a2dc-000c29675a27,001,4e8a0451-ee63-11e9-a2dc-000c29675a27,2020-12-30,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,10 days
31171,2020-12-24,0000015,f6548095-c106-11e0-8baa-00262d126e8b,000058022,f8f3fc71-6881-11e7-ab50-000c2945f18b,001,f8f3fc73-6881-11e7-ab50-000c2945f18b,2020-12-14,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,10 days
36756,2020-12-18,0000015,f6548095-c106-11e0-8baa-00262d126e8b,000183902,5c22fc9d-f726-11ea-ab3d-000c29a0142b,0000000001,5c22fc9f-f726-11ea-ab3d-000c29a0142b,2020-12-13,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,5 days


In [227]:
# Выберем только количество 
# дней в сроках поставок
lst = []
for item in msk_chel.difference:
    lst.append(item.days)
msk_chel['delivery'] = lst

msk_chel.sample(5)

C:\Users\2\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,date_receipt,warehouse_id,warehouse_guid,item_id,item_guid,item_charac_id,item_charac_guid,date_orders,fabricator_id,fabricator_guid,IsFromSupplier,difference,delivery
33606,2021-01-16,0000015,f6548095-c106-11e0-8baa-00262d126e8b,000136705,b78ef70b-14e9-11ea-a2ec-00505607eae9,003,1e8ea5a3-14ec-11ea-a2ec-00505607eae9,2021-01-04,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,12 days,12
36732,2020-12-26,0000015,f6548095-c106-11e0-8baa-00262d126e8b,000147397,db2125ef-4c12-11ea-a306-00505607eae9,0000000001,db2125f1-4c12-11ea-a306-00505607eae9,2020-12-19,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,7 days,7
22914,2020-10-30,0000015,f6548095-c106-11e0-8baa-00262d126e8b,000068516,a8c5f545-0a65-11e8-ab61-000c2945f18b,001,a8c5f54b-0a65-11e8-ab61-000c2945f18b,2020-10-21,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,9 days,9
27834,2020-11-05,0000015,f6548095-c106-11e0-8baa-00262d126e8b,000043693,85a1be0f-7eea-11e6-80d2-00155d000e07,001,85a1be11-7eea-11e6-80d2-00155d000e07,2020-10-24,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,12 days,12
34323,2020-12-17,0000015,f6548095-c106-11e0-8baa-00262d126e8b,000136737,df1323b8-14f3-11ea-a2ec-00505607eae9,001,07db7111-14f4-11ea-a2ec-00505607eae9,2020-11-28,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,19 days,19


In [229]:
deliv_10 = msk_chel.groupby(
    ['warehouse_id', 'warehouse_guid', 'fabricator_guid', 'IsFromSupplier']
).agg({'delivery': percentile(10)}).reset_index()
deliv_10['10'] = deliv_10.delivery
deliv_10.drop('delivery', axis=1, inplace=True)
deliv_10

,warehouse_id,warehouse_guid,fabricator_guid,IsFromSupplier,10
0,0000015,f6548095-c106-11e0-8baa-00262d126e8b,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,7


In [230]:
deliv_20 = msk_chel.groupby(
    ['warehouse_id', 'warehouse_guid', 'fabricator_guid', 'IsFromSupplier']
).agg({'delivery': percentile(20)}).reset_index()
deliv_20['20'] = deliv_20.delivery
deliv_20.drop('delivery', axis=1, inplace=True)
deliv_20

,warehouse_id,warehouse_guid,fabricator_guid,IsFromSupplier,20
0,0000015,f6548095-c106-11e0-8baa-00262d126e8b,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,7


In [231]:
deliv_30 = msk_chel.groupby(
    ['warehouse_id', 'warehouse_guid', 'fabricator_guid', 'IsFromSupplier']
).agg({'delivery': percentile(30)}).reset_index()
deliv_30['30'] = deliv_30.delivery
deliv_30.drop('delivery', axis=1, inplace=True)
deliv_30

,warehouse_id,warehouse_guid,fabricator_guid,IsFromSupplier,30
0,0000015,f6548095-c106-11e0-8baa-00262d126e8b,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,9


In [232]:
deliv_40 = msk_chel.groupby(
    ['warehouse_id', 'warehouse_guid', 'fabricator_guid', 'IsFromSupplier']
).agg({'delivery': percentile(40)}).reset_index()
deliv_40['40'] = deliv_40.delivery
deliv_40.drop('delivery', axis=1, inplace=True)
deliv_40

,warehouse_id,warehouse_guid,fabricator_guid,IsFromSupplier,40
0,0000015,f6548095-c106-11e0-8baa-00262d126e8b,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,9


In [233]:
deliv_50 = msk_chel.groupby(
    ['warehouse_id', 'warehouse_guid', 'fabricator_guid', 'IsFromSupplier']
).agg({'delivery': percentile(50)}).reset_index()
deliv_50['50'] = deliv_50.delivery
deliv_50.drop('delivery', axis=1, inplace=True)
deliv_50

,warehouse_id,warehouse_guid,fabricator_guid,IsFromSupplier,50
0,0000015,f6548095-c106-11e0-8baa-00262d126e8b,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,10


In [234]:
deliv_60 = msk_chel.groupby(
    ['warehouse_id', 'warehouse_guid', 'fabricator_guid', 'IsFromSupplier']
).agg({'delivery': percentile(60)}).reset_index()
deliv_60['60'] = deliv_60.delivery
deliv_60.drop('delivery', axis=1, inplace=True)
deliv_60

,warehouse_id,warehouse_guid,fabricator_guid,IsFromSupplier,60
0,0000015,f6548095-c106-11e0-8baa-00262d126e8b,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,12


In [235]:
deliv_70 = msk_chel.groupby(
    ['warehouse_id', 'warehouse_guid', 'fabricator_guid', 'IsFromSupplier']
).agg({'delivery': percentile(70)}).reset_index()
deliv_70['70'] = deliv_70.delivery
deliv_70.drop('delivery', axis=1, inplace=True)
deliv_70

,warehouse_id,warehouse_guid,fabricator_guid,IsFromSupplier,70
0,0000015,f6548095-c106-11e0-8baa-00262d126e8b,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,14


In [236]:
deliv_80 = msk_chel.groupby(
    ['warehouse_id', 'warehouse_guid', 'fabricator_guid', 'IsFromSupplier']
).agg({'delivery': percentile(80)}).reset_index()
deliv_80['80'] = deliv_80.delivery
deliv_80.drop('delivery', axis=1, inplace=True)
deliv_80

,warehouse_id,warehouse_guid,fabricator_guid,IsFromSupplier,80
0,0000015,f6548095-c106-11e0-8baa-00262d126e8b,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,20


In [237]:
deliv_90 = msk_chel.groupby(
    ['warehouse_id', 'warehouse_guid', 'fabricator_guid', 'IsFromSupplier']
).agg({'delivery': percentile(90)}).reset_index()
deliv_90['90'] = deliv_90.delivery
deliv_90.drop('delivery', axis=1, inplace=True)
deliv_90

,warehouse_id,warehouse_guid,fabricator_guid,IsFromSupplier,90
0,0000015,f6548095-c106-11e0-8baa-00262d126e8b,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,32


In [238]:
deliv_100 = msk_chel.groupby(
    ['warehouse_id', 'warehouse_guid', 'fabricator_guid', 'IsFromSupplier']
).agg({'delivery': percentile(100)}).reset_index()
deliv_100['100'] = deliv_100.delivery
deliv_100.drop('delivery', axis=1, inplace=True)
deliv_100

,warehouse_id,warehouse_guid,fabricator_guid,IsFromSupplier,100
0,0000015,f6548095-c106-11e0-8baa-00262d126e8b,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,182


In [239]:
m10_20 = pd.merge(
        deliv_10, deliv_20,
        on=['warehouse_id', 'warehouse_guid', 'fabricator_guid', 'IsFromSupplier'],
        how='left'
    )
m10_20

,warehouse_id,warehouse_guid,fabricator_guid,IsFromSupplier,10,20
0,0000015,f6548095-c106-11e0-8baa-00262d126e8b,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,7,7


In [240]:
m30_40 = pd.merge(
        deliv_30, deliv_40,
        on=['warehouse_id', 'warehouse_guid', 'fabricator_guid', 'IsFromSupplier'],
        how='left'
    )
m30_40

,warehouse_id,warehouse_guid,fabricator_guid,IsFromSupplier,30,40
0,0000015,f6548095-c106-11e0-8baa-00262d126e8b,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,9,9


In [241]:
m50_60 = pd.merge(
        deliv_50, deliv_60,
        on=['warehouse_id', 'warehouse_guid', 'fabricator_guid', 'IsFromSupplier'],
        how='left'
    )
m50_60

,warehouse_id,warehouse_guid,fabricator_guid,IsFromSupplier,50,60
0,0000015,f6548095-c106-11e0-8baa-00262d126e8b,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,10,12


In [242]:
m70_80 = pd.merge(
        deliv_70, deliv_80,
        on=['warehouse_id', 'warehouse_guid', 'fabricator_guid', 'IsFromSupplier'],
        how='left'
    )
m70_80

,warehouse_id,warehouse_guid,fabricator_guid,IsFromSupplier,70,80
0,0000015,f6548095-c106-11e0-8baa-00262d126e8b,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,14,20


In [243]:
m90_100 = pd.merge(
        deliv_90, deliv_100,
        on=['warehouse_id', 'warehouse_guid', 'fabricator_guid', 'IsFromSupplier'],
        how='left'
    )
m90_100

,warehouse_id,warehouse_guid,fabricator_guid,IsFromSupplier,90,100
0,0000015,f6548095-c106-11e0-8baa-00262d126e8b,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,32,182


In [244]:
m10_40 = pd.merge(
        m10_20, m30_40,
        on=['warehouse_id', 'warehouse_guid', 'fabricator_guid', 'IsFromSupplier'],
        how='left'
    )
m10_40

,warehouse_id,warehouse_guid,fabricator_guid,IsFromSupplier,10,20,30,40
0,0000015,f6548095-c106-11e0-8baa-00262d126e8b,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,7,7,9,9


In [245]:
m50_80 = pd.merge(
        m50_60, m70_80,
        on=['warehouse_id', 'warehouse_guid', 'fabricator_guid', 'IsFromSupplier'],
        how='left'
    )
m50_80

,warehouse_id,warehouse_guid,fabricator_guid,IsFromSupplier,50,60,70,80
0,0000015,f6548095-c106-11e0-8baa-00262d126e8b,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,10,12,14,20


In [246]:
m50_100 = pd.merge(
        m50_80, m90_100,
        on=['warehouse_id', 'warehouse_guid', 'fabricator_guid', 'IsFromSupplier'],
        how='left'
    )
m50_100

,warehouse_id,warehouse_guid,fabricator_guid,IsFromSupplier,50,60,70,80,90,100
0,0000015,f6548095-c106-11e0-8baa-00262d126e8b,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,10,12,14,20,32,182


In [247]:
msk_chel_per = pd.merge(
        m10_40, m50_100,
        on=['warehouse_id', 'warehouse_guid', 'fabricator_guid', 'IsFromSupplier'],
        how='left'
    )
msk_chel_per

,warehouse_id,warehouse_guid,fabricator_guid,IsFromSupplier,10,20,30,40,50,60,70,80,90,100
0,0000015,f6548095-c106-11e0-8baa-00262d126e8b,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,7,7,9,9,10,12,14,20,32,182


In [249]:
msk_chel_per.to_excel('percentile_msk_chel.xlsx', index=False)  

## 2. Что то другое

In [74]:
delivery = pd.DataFrame(load_data_from_api(url))

In [75]:
delivery

,date_receipt,warehouse_id,warehouse_guid,item_id,item_guid,item_charac_id,item_charac_guid,date_orders,fabricator_id,fabricator_guid
0,2021-01-11T00:00:00,0000045,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,000044859,a39a1cf3-a0c1-11e6-80d2-00155d000e07,017,29885384-b4a1-11e7-ab58-000c2945f18b,2020-12-31T00:00:00,ÐÐ0000015,f6548095-c106-11e0-8baa-00262d126e8b
1,2021-01-11T00:00:00,0000045,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,000105054,bbda3887-3ffc-11e9-a2d4-000c29675a27,001,c2a1ef3a-3ffc-11e9-a2d4-000c29675a27,2020-12-27T00:00:00,ÐÐ0000015,f6548095-c106-11e0-8baa-00262d126e8b
2,2021-01-11T00:00:00,0000045,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,000044852,48dbf4f8-a0bf-11e6-80d2-00155d000e07,021,95fd1c85-b49a-11e7-ab58-000c2945f18b,2020-12-31T00:00:00,ÐÐ0000015,f6548095-c106-11e0-8baa-00262d126e8b
3,2021-01-11T00:00:00,0000045,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,000044849,48dbf4e3-a0bf-11e6-80d2-00155d000e07,065,6e20c993-dd95-4c92-b70e-a4bd33d18e80,2020-12-31T00:00:00,ÐÐ0000015,f6548095-c106-11e0-8baa-00262d126e8b
4,2021-01-11T00:00:00,0000045,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,000044848,48dbf4dc-a0bf-11e6-80d2-00155d000e07,071,aa0fc549-e38a-401e-8031-4ccf8f2f35bc,2020-12-31T00:00:00,ÐÐ0000015,f6548095-c106-11e0-8baa-00262d126e8b
...,...,...,...,...,...,...,...,...,...,...
1841,2021-01-12T00:00:00,0000123,4bc4d898-ec0e-11e9-a2dc-000c29675a27,000045549,1cfff946-b5a7-11e6-80d4-00155d000e07,001,1cfff94a-b5a7-11e6-80d4-00155d000e07,2021-01-09T00:00:00,ÐÐ0000015,f6548095-c106-11e0-8baa-00262d126e8b
1842,2021-01-12T00:00:00,0000123,4bc4d898-ec0e-11e9-a2dc-000c29675a27,000129511,5994f93e-f006-11e9-a2dc-000c29675a27,002,5994f941-f006-11e9-a2dc-000c29675a27,2021-01-09T00:00:00,ÐÐ0000015,f6548095-c106-11e0-8baa-00262d126e8b
1843,2021-01-12T00:00:00,0000123,4bc4d898-ec0e-11e9-a2dc-000c29675a27,000063684,cfce35fa-d363-11e7-ab61-000c2945f18b,004,99e00703-d368-11e7-ab61-000c2945f18b,2021-01-10T00:00:00,ÐÐ0000015,f6548095-c106-11e0-8baa-00262d126e8b
1844,2021-01-12T00:00:00,0000123,4bc4d898-ec0e-11e9-a2dc-000c29675a27,000044851,48dbf4f1-a0bf-11e6-80d2-00155d000e07,067,d254c3d5-9309-4fa9-9e1f-dedbe64ef735,2020-12-29T00:00:00,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b


In [58]:
delivery.fabricator_guid.nunique()

35

In [59]:
delivery.warehouse_guid.nunique()

7

In [60]:
ekb = delivery.loc[(delivery['fabricator_guid'] == '5f219248-fadd-11df-8a53-00262d126e8b') & (delivery['warehouse_guid'] == '5f219248-fadd-11df-8a53-00262d126e8b')]

In [63]:
ekb.sample(5)

,date_receipt,warehouse_id,warehouse_guid,item_id,item_guid,item_charac_id,item_charac_guid,date_orders,fabricator_id,fabricator_guid
12807,2020-11-19T00:00:00,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000152956,050d0a23-6363-11ea-a307-00505607eae9,0000000001,19641c4a-6363-11ea-a307-00505607eae9,2020-11-15T00:00:00,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b
13950,2020-11-27T00:00:00,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000006299,2607b546-49a9-11e2-9140-f46d04736e4a,040,d21f3276-2c01-11e8-ab63-000c2945f18b,2020-11-24T00:00:00,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b
12206,2020-11-19T00:00:00,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000158801,da28e7ec-83bf-11ea-ab36-000c29a01421,0000000010,c6c3d4f0-e148-4001-825c-2b5ce95bb599,2020-11-11T00:00:00,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b
18055,2020-12-29T00:00:00,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000042746,d4e87115-6a7c-11e6-80d2-00155d000e07,034,380f690b-7550-11e9-a2d6-000c29675a31,2020-12-23T00:00:00,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b
17367,2020-12-21T00:00:00,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000043441,dd1d0bd1-7a33-11e6-80d2-00155d000e07,001,dd1d0bd3-7a33-11e6-80d2-00155d000e07,2020-12-16T00:00:00,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b


In [64]:
object_to_date(ekb, ekb.date_receipt, 'date_receipt')
object_to_date(ekb, ekb.date_orders, 'date_orders')

C:\Users\2\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [65]:
ekb

,date_receipt,warehouse_id,warehouse_guid,item_id,item_guid,item_charac_id,item_charac_guid,date_orders,fabricator_id,fabricator_guid
2693,2020-11-07,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000158855,6559fd67-8457-11ea-ab36-000c29a01421,0000000007,2793f287-3304-436f-bb06-e97ef84c727a,2020-09-15,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b
2887,2020-11-25,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000143592,91da928f-3b62-11ea-a2fd-00505607eae9,0000000001,9cd60e8f-3b62-11ea-a2fd-00505607eae9,2020-11-11,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b
2890,2020-11-26,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000143592,91da928f-3b62-11ea-a2fd-00505607eae9,0000000001,9cd60e8f-3b62-11ea-a2fd-00505607eae9,2020-11-11,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b
4015,2020-10-14,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000168447,33d9ced1-b12a-11ea-ab38-000c29a01421,0000000001,3b357f33-b12a-11ea-ab38-000c29a01421,2020-09-26,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b
4324,2020-12-08,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000044849,48dbf4e3-a0bf-11e6-80d2-00155d000e07,015,0283eee3-6ad3-11e7-ab50-000c2945f18b,2020-11-30,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b
...,...,...,...,...,...,...,...,...,...,...
17785,2020-12-25,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000075274,d89cfc03-43a0-11e8-a2bb-000c29dcdf3d,001,d89cfc06-43a0-11e8-a2bb-000c29dcdf3d,2020-12-21,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b
18055,2020-12-29,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000042746,d4e87115-6a7c-11e6-80d2-00155d000e07,034,380f690b-7550-11e9-a2d6-000c29675a31,2020-12-23,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b
18057,2020-12-29,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000077877,03434da2-5ce0-11e8-a2c3-000c29dcdf3d,001,89736aae-5cee-11e8-a2c3-000c29dcdf3d,2020-12-23,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b
18081,2020-12-29,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000145107,049321d8-425c-11ea-a302-00505607eae9,0000000017,ae3422f5-e3fe-4dbf-868d-018adc41ae78,2020-12-23,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b


In [66]:
# Создадим предиктор со сроками поставок
ekb['difference'] = ekb.date_receipt - ekb.date_orders

# Посмотрим результат
ekb.head(10)

C:\Users\2\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,date_receipt,warehouse_id,warehouse_guid,item_id,item_guid,item_charac_id,item_charac_guid,date_orders,fabricator_id,fabricator_guid,difference
2693,2020-11-07,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000158855,6559fd67-8457-11ea-ab36-000c29a01421,0000000007,2793f287-3304-436f-bb06-e97ef84c727a,2020-09-15,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b,53 days
2887,2020-11-25,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000143592,91da928f-3b62-11ea-a2fd-00505607eae9,0000000001,9cd60e8f-3b62-11ea-a2fd-00505607eae9,2020-11-11,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b,14 days
2890,2020-11-26,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000143592,91da928f-3b62-11ea-a2fd-00505607eae9,0000000001,9cd60e8f-3b62-11ea-a2fd-00505607eae9,2020-11-11,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b,15 days
4015,2020-10-14,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000168447,33d9ced1-b12a-11ea-ab38-000c29a01421,0000000001,3b357f33-b12a-11ea-ab38-000c29a01421,2020-09-26,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b,18 days
4324,2020-12-08,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000044849,48dbf4e3-a0bf-11e6-80d2-00155d000e07,015,0283eee3-6ad3-11e7-ab50-000c2945f18b,2020-11-30,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b,8 days
4623,2020-10-16,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000121369,f6e24d6f-bf3a-11e9-a2d7-000c29675a27,002,97bc243e-c0ce-11e9-a2d7-000c29675a27,2020-10-01,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b,15 days
4998,2020-10-16,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000063395,3cda53e6-cf50-11e7-ab61-000c2945f18b,002,f564ea81-d501-11e7-ab61-000c2945f18b,2020-10-03,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b,13 days
6273,2020-10-30,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000044765,3578a482-9f67-11e6-80d2-00155d000e07,001,3578a484-9f67-11e6-80d2-00155d000e07,2020-10-09,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b,21 days
6476,2020-10-15,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000158561,b79585a9-82f6-11ea-ab36-000c29a01421,0000000002,e1e37ad6-b6d3-4f20-8053-3ec4ef0066fd,2020-10-11,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b,4 days
7107,2020-11-05,0000007,5f219248-fadd-11df-8a53-00262d126e8b,000058496,5f86fe9a-6dd7-11e7-ab50-000c2945f18b,001,5f86fe9e-6dd7-11e7-ab50-000c2945f18b,2020-10-14,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b,22 days


In [67]:
# Выберем только количество 
# дней в сроках поставок
lst = []
for item in ekb.difference:
    lst.append(item.days)
ekb['delivery'] = lst

C:\Users\2\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [72]:
ekb.delivery.values

array([53, 14, 15, 18,  8, 15, 13, 21,  4, 22,  5,  3,  9,  7, 19, 10, 15,
        6, 59,  6,  8,  8,  6,  9,  7,  8,  6, 14,  7,  4,  4,  6,  4,  6,
        5,  5,  3,  3,  3, 29,  9,  5, 14,  5,  7,  4,  6,  5,  6, 11,  4,
        4, 12, 10,  8,  7,  8,  8,  5,  7,  7,  5,  5,  4,  6,  6,  6,  4],
      dtype=int64)

In [ ]:
# # Сгруппируем датасет по производителям и складам, агрегирую сроки поставки процентилем
deliv_min = ekb.groupby(['PlacementFromId', 'PlacementToId', 'IsFromSupplier']).agg(
    {'delivery': percentile(50)}).reset_index()
deliv_max = ekb.groupby(['PlacementFromId', 'PlacementToId', 'IsFromSupplier']).agg(
    {'delivery': percentile(90)}).reset_index()

In [ ]:
deliv_max['DaysTo'] = deliv_max.delivery
deliv_min['DaysFrom'] = deliv_min.delivery
deliv_max.drop('delivery', axis=1, inplace=True)
deliv_min.drop('delivery', axis=1, inplace=True)

In [52]:
# Посмотрим локальную директорию
os.listdir('C:/Users/2/PycharmProjects/delivery_time_statistics/datasets')

['delivery_time.csv', 'graphics.csv']

In [53]:
# Загрузим датасет
deliv = pd.read_csv('/Users/2/PycharmProjects/delivery_time_statistics/datasets/delivery_time.csv')

C:\Users\2\anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [54]:
# Проверим 5 рандомных записей
deliv.sample(5)

,date_receipt,warehouse_id,warehouse_guid,item_id,item_guid,item_charac_id,item_charac_guid,date_orders,fabricator_id,fabricator_guid,IsFromSupplier
10706,2020-11-26T00:00:00,7,5f219248-fadd-11df-8a53-00262d126e8b,117662,782d385a-9bf3-11e9-a2d7-000c29675a27,1.0,782d385c-9bf3-11e9-a2d7-000c29675a27,2020-11-19T00:00:00,ÐÐ0000015,f6548095-c106-11e0-8baa-00262d126e8b,0
9151,2020-11-06T00:00:00,7,5f219248-fadd-11df-8a53-00262d126e8b,58335,70874380-6c65-11e7-ab50-000c2945f18b,1.0,70874382-6c65-11e7-ab50-000c2945f18b,2020-10-30T00:00:00,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0
10967,2020-12-31T00:00:00,7,5f219248-fadd-11df-8a53-00262d126e8b,161860,0b6b1e7e-9617-11ea-ab38-000c29a01421,1.0,0b6b1e80-9617-11ea-ab38-000c29a01421,2020-12-27T00:00:00,ÐÐ0000015,f6548095-c106-11e0-8baa-00262d126e8b,0
44865,2020-10-24T00:00:00,45,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,67980,185b0892-0651-11e8-ab61-000c2945f18b,2.0,185b0899-0651-11e8-ab61-000c2945f18b,2020-10-13T00:00:00,ÐÐ0000007,5f219248-fadd-11df-8a53-00262d126e8b,0
21531,2020-11-16T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,44806,a9cccfa3-a0b7-11e6-80d2-00155d000e07,1.0,a9cccfa5-a0b7-11e6-80d2-00155d000e07,2020-11-06T00:00:00,ÐÐ0000123,4bc4d898-ec0e-11e9-a2dc-000c29675a27,0


In [58]:
msk_ekb = deliv.loc[(deliv['fabricator_guid'] == 'a2f89531-75de-11e8-a2c3-000c29dcdf3d') & (deliv['warehouse_guid'] == '5f219248-fadd-11df-8a53-00262d126e8b')]

In [26]:
msk_ekb.drop(['item_id', 'item_guid', 'item_charac_id', 'item_charac_guid'], axis=1, inplace=True)

C:\Users\2\anaconda3\envs\tf\lib\site-packages\pandas\core\frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [59]:
msk_ekb

,date_receipt,warehouse_id,warehouse_guid,item_id,item_guid,item_charac_id,item_charac_guid,date_orders,fabricator_id,fabricator_guid,IsFromSupplier
0,2020-10-21T00:00:00,7,5f219248-fadd-11df-8a53-00262d126e8b,129111,ad818131-ee77-11e9-a2dc-000c29675a27,8.0,a2f3bc1e-d22a-41f3-8ab6-0942d1cc411b,2020-09-23T00:00:00,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0
4,2020-10-30T00:00:00,7,5f219248-fadd-11df-8a53-00262d126e8b,135901,98d2b397-10f6-11ea-a2e9-00505607eae9,2.0,5faa48df-95ba-4493-9e23-f8c95ad4b55b,2020-06-17T00:00:00,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0
5,2020-10-22T00:00:00,7,5f219248-fadd-11df-8a53-00262d126e8b,158293,031bc29c-8075-11ea-ab36-000c29a01421,2.0,7fcc0431-f494-4974-8bd6-ee87fead0ee5,2020-07-18T00:00:00,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0
8,2020-10-30T00:00:00,7,5f219248-fadd-11df-8a53-00262d126e8b,160530,72d58911-8c9c-11ea-ab36-000c29a01421,1.0,72d58913-8c9c-11ea-ab36-000c29a01421,2020-08-10T00:00:00,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0
9,2020-10-30T00:00:00,7,5f219248-fadd-11df-8a53-00262d126e8b,124718,0c6385bd-d478-11e9-a2d7-000c29675a27,3.0,346f12b6-6e44-42d4-b405-da74b04bdac9,2020-06-12T00:00:00,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0
...,...,...,...,...,...,...,...,...,...,...,...
19445,2021-01-18T00:00:00,7,5f219248-fadd-11df-8a53-00262d126e8b,103777,5819d540-35a5-11e9-a2ce-000c29675a27,6.0,6b176cf7-d553-11e9-a2d7-000c29675a27,2021-01-12T00:00:00,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0
19452,2021-01-18T00:00:00,7,5f219248-fadd-11df-8a53-00262d126e8b,71353,1039151b-267f-11e8-ab63-000c2945f18b,1.0,1039151f-267f-11e8-ab63-000c2945f18b,2021-01-12T00:00:00,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0
19453,2021-01-18T00:00:00,7,5f219248-fadd-11df-8a53-00262d126e8b,61381,884e7b8c-a985-11e7-ab54-000c2945f18b,1.0,884e7b92-a985-11e7-ab54-000c2945f18b,2021-01-12T00:00:00,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0
19455,2021-01-18T00:00:00,7,5f219248-fadd-11df-8a53-00262d126e8b,39762,556331d5-1767-11e6-80cc-00155d000e07,1.0,556331d7-1767-11e6-80cc-00155d000e07,2021-01-12T00:00:00,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0


In [60]:
object_to_date(msk_ekb, msk_ekb.date_receipt, 'date_receipt')
object_to_date(msk_ekb, msk_ekb.date_orders, 'date_orders')

C:\Users\2\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [61]:
msk_ekb['difference'] = msk_ekb.date_receipt - msk_ekb.date_orders
msk_ekb.sample(10)

C:\Users\2\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,date_receipt,warehouse_id,warehouse_guid,item_id,item_guid,item_charac_id,item_charac_guid,date_orders,fabricator_id,fabricator_guid,IsFromSupplier,difference
18051,2021-01-16,7,5f219248-fadd-11df-8a53-00262d126e8b,85072,26703944-9c5f-11e8-a2c0-00505607eae9,1.0,26703946-9c5f-11e8-a2c0-00505607eae9,2020-12-24,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,23 days
8315,2020-12-15,7,5f219248-fadd-11df-8a53-00262d126e8b,161524,68d55d24-9494-11ea-ab37-000c29a01421,2.0,68d55d27-9494-11ea-ab37-000c29a01421,2020-12-03,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,12 days
13985,2020-12-08,7,5f219248-fadd-11df-8a53-00262d126e8b,132080,83fa81ad-fbdb-11e9-a2e0-00505607eae9,1.0,c9a8aeb9-fbdb-11e9-a2e0-00505607eae9,2020-11-28,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,10 days
8150,2020-11-29,7,5f219248-fadd-11df-8a53-00262d126e8b,158563,3aa45693-82f7-11ea-ab36-000c29a01421,2.0,ab291e12-6b99-4a41-97a0-6ab9601f9ed5,2020-11-22,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,7 days
5374,2020-11-14,7,5f219248-fadd-11df-8a53-00262d126e8b,145546,f30ba8c3-43dd-11ea-a302-00505607eae9,1.0,f30ba8c5-43dd-11ea-a302-00505607eae9,2020-11-05,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,9 days
1505,2020-11-09,7,5f219248-fadd-11df-8a53-00262d126e8b,105039,1da795d4-3ffc-11e9-a2d4-000c29675a27,1.0,3b53c38d-3ffc-11e9-a2d4-000c29675a27,2020-09-11,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,59 days
19323,2021-01-18,7,5f219248-fadd-11df-8a53-00262d126e8b,171258,96e5e4b5-bc55-11ea-ab38-000c29a01421,5.0,d1f4cda8-1f0a-4949-bdc1-767682b73e35,2021-01-11,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,7 days
15871,2020-12-29,7,5f219248-fadd-11df-8a53-00262d126e8b,129759,bef9f1ed-f17f-11e9-a2dc-000c29675a27,1.0,bef9f1f0-f17f-11e9-a2dc-000c29675a27,2020-12-21,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,8 days
6924,2020-11-06,7,5f219248-fadd-11df-8a53-00262d126e8b,125799,4cc86719-dab8-11e9-a2d9-000c29675a27,1.0,4cc8671b-dab8-11e9-a2d9-000c29675a27,2020-10-31,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,6 days
6694,2020-10-30,7,5f219248-fadd-11df-8a53-00262d126e8b,163190,8ac2717a-9b68-11ea-ab38-000c29a01421,1.0,8ac2717c-9b68-11ea-ab38-000c29a01421,2020-10-19,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,11 days


In [64]:
msk_ekb.sample(10)

,date_receipt,warehouse_id,warehouse_guid,item_id,item_guid,item_charac_id,item_charac_guid,date_orders,fabricator_id,fabricator_guid,IsFromSupplier,difference
2389,2020-10-30,7,5f219248-fadd-11df-8a53-00262d126e8b,65553,e572ce3b-e930-11e7-ab61-000c2945f18b,1.0,e572ce3d-e930-11e7-ab61-000c2945f18b,2020-10-09,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,21 days
5876,2020-12-24,7,5f219248-fadd-11df-8a53-00262d126e8b,44851,48dbf4f1-a0bf-11e6-80d2-00155d000e07,25.0,bf73178d-4dec-11e8-a2c1-000c29dcdf3d,2020-12-16,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,8 days
10579,2021-01-16,7,5f219248-fadd-11df-8a53-00262d126e8b,113009,61b46e8d-7bb3-11e9-a2d6-000c29675a31,1.0,b35f4467-7bb3-11e9-a2d6-000c29675a31,2020-12-30,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,17 days
6113,2020-11-16,7,5f219248-fadd-11df-8a53-00262d126e8b,92130,effe88f6-d39d-11e8-a2c4-00505607eae9,1.0,1b6dd451-d39e-11e8-a2c4-00505607eae9,2020-11-07,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,9 days
9394,2020-11-06,7,5f219248-fadd-11df-8a53-00262d126e8b,94166,df5fcdd2-e35f-11e8-a2c4-00505607eae9,1.0,7505c8ce-e360-11e8-a2c4-00505607eae9,2020-11-01,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,5 days
8771,2020-11-06,7,5f219248-fadd-11df-8a53-00262d126e8b,56271,2dcc2b58-54e6-11e7-ab48-000c2945f18b,1.0,2dcc2b5a-54e6-11e7-ab48-000c2945f18b,2020-10-30,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,7 days
10389,2020-11-16,7,5f219248-fadd-11df-8a53-00262d126e8b,139142,1ac2007b-1e6b-11ea-a2f1-00505607eae9,4.0,5c3306be-263a-11ea-a2f6-00505607eae9,2020-11-06,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,10 days
3729,2021-01-14,7,5f219248-fadd-11df-8a53-00262d126e8b,8068,e7239912-8256-11e2-be7c-f46d04736e4a,8.0,3c42b3cf-825d-11e2-be7c-f46d04736e4a,2020-09-30,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,106 days
18338,2021-01-16,7,5f219248-fadd-11df-8a53-00262d126e8b,139125,5f126c00-1e61-11ea-a2f1-00505607eae9,3.0,d301f4f4-1e61-11ea-a2f1-00505607eae9,2020-12-26,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,21 days
16012,2020-12-23,7,5f219248-fadd-11df-8a53-00262d126e8b,8078,3c42b3f4-825d-11e2-be7c-f46d04736e4a,30.0,bd6a0abf-8260-11e2-be7c-f46d04736e4a,2020-12-09,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,14 days


In [65]:
msk_ekb.to_csv('msk_to_ekb.csv', index=False)  

In [66]:
msk_chel = deliv.loc[(deliv['fabricator_guid'] == 'a2f89531-75de-11e8-a2c3-000c29dcdf3d') & (deliv['warehouse_guid'] == 'f6548095-c106-11e0-8baa-00262d126e8b')]

In [33]:
msk_chel.drop(['item_id', 'item_guid', 'item_charac_id', 'item_charac_guid'], axis=1, inplace=True)

C:\Users\2\anaconda3\envs\tf\lib\site-packages\pandas\core\frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [67]:
object_to_date(msk_chel, msk_chel.date_receipt, 'date_receipt')
object_to_date(msk_chel, msk_chel.date_orders, 'date_orders')

C:\Users\2\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [68]:
msk_chel['difference'] = msk_chel.date_receipt - msk_chel.date_orders
msk_chel.sample(15)

C:\Users\2\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,date_receipt,warehouse_id,warehouse_guid,item_id,item_guid,item_charac_id,item_charac_guid,date_orders,fabricator_id,fabricator_guid,IsFromSupplier,difference
21020,2020-11-02,15,f6548095-c106-11e0-8baa-00262d126e8b,139142,1ac2007b-1e6b-11ea-a2f1-00505607eae9,6.0,a77c2652-263a-11ea-a2f6-00505607eae9,2020-10-23,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,10 days
20492,2020-11-05,15,f6548095-c106-11e0-8baa-00262d126e8b,152205,34609af3-5eb8-11ea-a307-00505607eae9,2.0,e474ead2-6a4a-4930-b6a0-cad7aaf234f3,2020-09-30,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,36 days
26920,2020-10-29,15,f6548095-c106-11e0-8baa-00262d126e8b,58336,70874384-6c65-11e7-ab50-000c2945f18b,1.0,70874386-6c65-11e7-ab50-000c2945f18b,2020-10-20,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,9 days
27737,2020-11-18,15,f6548095-c106-11e0-8baa-00262d126e8b,89677,cc511b11-c574-11e8-a2c1-00505607eae9,1.0,cc511b13-c574-11e8-a2c1-00505607eae9,2020-10-31,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,18 days
26564,2020-11-20,15,f6548095-c106-11e0-8baa-00262d126e8b,180742,a5878cdd-e904-11ea-ab3c-000c29a0142b,2.0,8887d25b-eb5d-11ea-ab3c-000c29a0142b,2020-11-12,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,8 days
28056,2020-11-23,15,f6548095-c106-11e0-8baa-00262d126e8b,155157,c617a749-6d9e-11ea-a307-00505607eae9,2.0,5fe40dfe-6da3-11ea-a307-00505607eae9,2020-11-14,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,9 days
31602,2020-12-05,15,f6548095-c106-11e0-8baa-00262d126e8b,93034,6dd9ff37-d8f3-11e8-a2c4-00505607eae9,1.0,8bffab6d-d8f3-11e8-a2c4-00505607eae9,2020-11-25,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,10 days
32397,2020-11-23,15,f6548095-c106-11e0-8baa-00262d126e8b,116629,3d9f6e49-93eb-11e9-a2d7-000c29675a27,1.0,487890af-93ed-11e9-a2d7-000c29675a27,2020-11-16,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,7 days
24162,2020-11-01,15,f6548095-c106-11e0-8baa-00262d126e8b,55533,208d3b98-46a9-11e7-ab48-000c2945f18b,5.0,464c75dd-46aa-11e7-ab48-000c2945f18b,2020-10-07,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,25 days
31509,2020-11-30,15,f6548095-c106-11e0-8baa-00262d126e8b,109652,d5d029d8-61b5-11e9-a2d6-000c29675a31,1.0,d5d029da-61b5-11e9-a2d6-000c29675a31,2020-11-25,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,5 days


In [69]:
msk_chel.sample(15)

,date_receipt,warehouse_id,warehouse_guid,item_id,item_guid,item_charac_id,item_charac_guid,date_orders,fabricator_id,fabricator_guid,IsFromSupplier,difference
29274,2020-11-18,15,f6548095-c106-11e0-8baa-00262d126e8b,155307,42b60f6e-6e62-11ea-a307-00505607eae9,2.0,8f1209f2-6e62-11ea-a307-00505607eae9,2020-11-02,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,16 days
29798,2020-11-20,15,f6548095-c106-11e0-8baa-00262d126e8b,39762,556331d5-1767-11e6-80cc-00155d000e07,2.0,556331d8-1767-11e6-80cc-00155d000e07,2020-11-14,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,6 days
20239,2020-11-14,15,f6548095-c106-11e0-8baa-00262d126e8b,137198,26215197-1672-11ea-a2ed-00505607eae9,1.0,500861bc-1672-11ea-a2ed-00505607eae9,2020-11-05,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,9 days
20931,2020-11-14,15,f6548095-c106-11e0-8baa-00262d126e8b,110835,504ea601-6825-11e9-a2d6-000c29675a31,1.0,504ea603-6825-11e9-a2d6-000c29675a31,2020-09-24,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,51 days
31959,2020-11-23,15,f6548095-c106-11e0-8baa-00262d126e8b,151084,b9e0ad17-5a2e-11ea-a307-00505607eae9,3.0,d605b343-5a2e-11ea-a307-00505607eae9,2020-11-14,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,9 days
38163,2021-01-09,15,f6548095-c106-11e0-8baa-00262d126e8b,42749,9f24429b-6a85-11e6-80d2-00155d000e07,37.0,74f6daf3-7551-11e9-a2d6-000c29675a31,2020-12-26,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,14 days
35465,2021-01-15,15,f6548095-c106-11e0-8baa-00262d126e8b,106895,ab26451f-4fcd-11e9-a2d5-000c29675a27,1.0,4bb294f9-4fce-11e9-a2d5-000c29675a27,2020-12-04,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,42 days
22451,2020-10-29,15,f6548095-c106-11e0-8baa-00262d126e8b,130174,0f0006e6-f3fe-11e9-a2dc-000c29675a27,3.0,1638e728-f3fe-11e9-a2dc-000c29675a27,2020-10-03,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,26 days
37158,2020-12-24,15,f6548095-c106-11e0-8baa-00262d126e8b,44879,8be53495-a0c6-11e6-80d2-00155d000e07,8.0,b7e15268-25c4-11e7-80d9-00155d000e07,2020-12-15,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,9 days
27756,2020-11-02,15,f6548095-c106-11e0-8baa-00262d126e8b,69870,9db39e2f-161f-11e8-ab61-000c2945f18b,1.0,9db39e34-161f-11e8-ab61-000c2945f18b,2020-10-25,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,8 days


In [70]:
msk_chel.to_csv('msk_to_chel.csv', index=False)

In [71]:
msk_ekb.head(15)

,date_receipt,warehouse_id,warehouse_guid,item_id,item_guid,item_charac_id,item_charac_guid,date_orders,fabricator_id,fabricator_guid,IsFromSupplier,difference
0,2020-10-21,7,5f219248-fadd-11df-8a53-00262d126e8b,129111,ad818131-ee77-11e9-a2dc-000c29675a27,8.0,a2f3bc1e-d22a-41f3-8ab6-0942d1cc411b,2020-09-23,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,28 days
4,2020-10-30,7,5f219248-fadd-11df-8a53-00262d126e8b,135901,98d2b397-10f6-11ea-a2e9-00505607eae9,2.0,5faa48df-95ba-4493-9e23-f8c95ad4b55b,2020-06-17,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,135 days
5,2020-10-22,7,5f219248-fadd-11df-8a53-00262d126e8b,158293,031bc29c-8075-11ea-ab36-000c29a01421,2.0,7fcc0431-f494-4974-8bd6-ee87fead0ee5,2020-07-18,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,96 days
8,2020-10-30,7,5f219248-fadd-11df-8a53-00262d126e8b,160530,72d58911-8c9c-11ea-ab36-000c29a01421,1.0,72d58913-8c9c-11ea-ab36-000c29a01421,2020-08-10,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,81 days
9,2020-10-30,7,5f219248-fadd-11df-8a53-00262d126e8b,124718,0c6385bd-d478-11e9-a2d7-000c29675a27,3.0,346f12b6-6e44-42d4-b405-da74b04bdac9,2020-06-12,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,140 days
14,2020-10-28,7,5f219248-fadd-11df-8a53-00262d126e8b,129384,5dd9d537-efcd-11e9-a2dc-000c29675a27,3.0,214c9046-65d6-4c2c-8b15-e0d46aa38701,2020-07-24,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,96 days
16,2020-10-30,7,5f219248-fadd-11df-8a53-00262d126e8b,119422,0d120534-ac42-11e9-a2d7-000c29675a27,4.0,0d120539-ac42-11e9-a2d7-000c29675a27,2020-07-04,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,118 days
19,2020-10-22,7,5f219248-fadd-11df-8a53-00262d126e8b,154177,e6795e21-684b-11ea-a307-00505607eae9,1.0,0828fbed-684c-11ea-a307-00505607eae9,2020-09-04,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,48 days
27,2020-10-22,7,5f219248-fadd-11df-8a53-00262d126e8b,126337,7fc7ff73-de98-11e9-a2d9-000c29675a27,1.0,a827765c-de98-11e9-a2d9-000c29675a27,2020-09-18,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,34 days
34,2020-10-22,7,5f219248-fadd-11df-8a53-00262d126e8b,77271,fd46fc38-5748-11e8-a2c3-000c29dcdf3d,2.0,fd46fc3b-5748-11e8-a2c3-000c29dcdf3d,2020-08-24,ÐÐ0000060,a2f89531-75de-11e8-a2c3-000c29dcdf3d,0,59 days


In [ ]:
deliv['B'] = np.minimum(deliv.A, deliv.A)

In [55]:
# Дропним ненужные фичи и переменуем нужные
deliv['OFFER_ID'] = deliv[['item_guid', 'item_charac_guid']].apply(lambda x: '#'.join(x), axis=1)
deliv.drop(list_to_drop, axis=1, inplace=True)
deliv.rename(
    columns={
        'warehouse_guid': 'PlacementToId',
        'fabricator_guid': 'PlacementFromId'
    }, inplace=True
)

In [56]:
# Посмотрим число уникальных IsFromSupplier
deliv.IsFromSupplier.unique()

array([0, 1], dtype=int64)

In [57]:
# Выберем только IsFromSupplier == 1
deliv = deliv.loc[deliv['IsFromSupplier'] == 1]

In [58]:
# Посмотрим число уникальных IsFromSupplier
deliv.IsFromSupplier.unique()

array([1], dtype=int64)

In [59]:
# Посмотрим на результат
deliv.sample(10)

,date_receipt,warehouse_id,PlacementToId,date_orders,PlacementFromId,IsFromSupplier,OFFER_ID
1168,2020-12-23T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-25T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...
1062,2020-12-23T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-16T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,a9cccfac-a0b7-11e6-80d2-00155d000e07#a9cccfae-...
682,2020-12-23T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-12-16T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,a9cccf91-a0b7-11e6-80d2-00155d000e07#a9cccf93-...
801,2020-12-23T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-20T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,a9cccf9a-a0b7-11e6-80d2-00155d000e07#a9cccf9c-...
1094,2020-12-23T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-28T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,775a465a-3ffc-11e9-a2d4-000c29675a27#7d65a253-...
1437,2020-12-23T00:00:00,7,5f219248-fadd-11df-8a53-00262d126e8b,2020-11-10T00:00:00,1f688887-b7fe-11e8-a2c1-00505607eae9,1,b2534b05-c6d1-11e8-a2c1-00505607eae9#d6929e5c-...
1012,2020-12-23T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-12-07T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,a9cccfa3-a0b7-11e6-80d2-00155d000e07#a9cccfa5-...
744,2020-12-23T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-12-05T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,a9cccf94-a0b7-11e6-80d2-00155d000e07#a9cccf96-...
996,2020-12-23T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-12-01T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,a9cccfa3-a0b7-11e6-80d2-00155d000e07#a9cccfa5-...
859,2020-12-23T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-12-05T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,a9cccf9a-a0b7-11e6-80d2-00155d000e07#a9cccf9c-...


In [60]:
deliv.PlacementFromId.nunique()

71

In [61]:
cur_warehouse = deliv.loc[deliv['PlacementFromId'] == '1f688887-b7fe-11e8-a2c1-00505607eae9']
cur_warehouse

,date_receipt,warehouse_id,PlacementToId,date_orders,PlacementFromId,IsFromSupplier,OFFER_ID
1432,2020-12-23T00:00:00,7,5f219248-fadd-11df-8a53-00262d126e8b,2020-10-18T00:00:00,1f688887-b7fe-11e8-a2c1-00505607eae9,1,c7d2abbe-f294-11ea-ab3c-000c29a0142b#c7d2abc0-...
1437,2020-12-23T00:00:00,7,5f219248-fadd-11df-8a53-00262d126e8b,2020-11-10T00:00:00,1f688887-b7fe-11e8-a2c1-00505607eae9,1,b2534b05-c6d1-11e8-a2c1-00505607eae9#d6929e5c-...


In [29]:
# Из типа объект делаем даты
object_to_date(cur_warehouse, cur_warehouse.date_receipt, 'date_receipt')
object_to_date(cur_warehouse, cur_warehouse.date_orders, 'date_orders')

C:\Users\2\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [30]:
# Создадим предиктор со сроками поставок
cur_warehouse['difference'] = cur_warehouse.date_receipt - cur_warehouse.date_orders

# Посмотрим результат
cur_warehouse

C:\Users\2\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,date_receipt,PlacementToId,date_orders,PlacementFromId,IsFromSupplier,OFFER_ID,difference
1432,2020-12-23,5f219248-fadd-11df-8a53-00262d126e8b,2020-10-18,1f688887-b7fe-11e8-a2c1-00505607eae9,1,c7d2abbe-f294-11ea-ab3c-000c29a0142b#c7d2abc0-...,66 days
1437,2020-12-23,5f219248-fadd-11df-8a53-00262d126e8b,2020-11-10,1f688887-b7fe-11e8-a2c1-00505607eae9,1,b2534b05-c6d1-11e8-a2c1-00505607eae9#d6929e5c-...,43 days


In [31]:
cur_warehouse

,date_receipt,PlacementToId,date_orders,PlacementFromId,IsFromSupplier,OFFER_ID,difference
1432,2020-12-23,5f219248-fadd-11df-8a53-00262d126e8b,2020-10-18,1f688887-b7fe-11e8-a2c1-00505607eae9,1,c7d2abbe-f294-11ea-ab3c-000c29a0142b#c7d2abc0-...,66 days
1437,2020-12-23,5f219248-fadd-11df-8a53-00262d126e8b,2020-11-10,1f688887-b7fe-11e8-a2c1-00505607eae9,1,b2534b05-c6d1-11e8-a2c1-00505607eae9#d6929e5c-...,43 days


In [32]:
# Выберем только количество 
# дней в сроках поставок
lst = []
for item in cur_warehouse.difference:
    lst.append(item.days)
cur_warehouse['delivery'] = lst

cur_warehouse

C:\Users\2\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,date_receipt,PlacementToId,date_orders,PlacementFromId,IsFromSupplier,OFFER_ID,difference,delivery
1432,2020-12-23,5f219248-fadd-11df-8a53-00262d126e8b,2020-10-18,1f688887-b7fe-11e8-a2c1-00505607eae9,1,c7d2abbe-f294-11ea-ab3c-000c29a0142b#c7d2abc0-...,66 days,66
1437,2020-12-23,5f219248-fadd-11df-8a53-00262d126e8b,2020-11-10,1f688887-b7fe-11e8-a2c1-00505607eae9,1,b2534b05-c6d1-11e8-a2c1-00505607eae9#d6929e5c-...,43 days,43


In [33]:
deliv_min = cur_warehouse.groupby(['PlacementFromId', 'PlacementToId']).agg(
    {'delivery': percentile(50)}).reset_index()

In [36]:
deliv_max = cur_warehouse.groupby(['PlacementFromId', 'PlacementToId', 'IsFromSupplier']).agg(
    {'delivery': percentile(90)}).reset_index()

deliv_max

,PlacementFromId,PlacementToId,IsFromSupplier,delivery
0,1f688887-b7fe-11e8-a2c1-00505607eae9,5f219248-fadd-11df-8a53-00262d126e8b,1,63.7


In [37]:
deliv_min

,PlacementFromId,PlacementToId,delivery
0,1f688887-b7fe-11e8-a2c1-00505607eae9,5f219248-fadd-11df-8a53-00262d126e8b,54.5


In [40]:
deliv_mean = cur_warehouse.groupby(['PlacementFromId', 'PlacementToId']).agg({'delivery': np.mean}).reset_index()
deliv_mean

,PlacementFromId,PlacementToId,delivery
0,1f688887-b7fe-11e8-a2c1-00505607eae9,5f219248-fadd-11df-8a53-00262d126e8b,54.5


In [62]:
# Отсортируем записи по дате поступления на склад
deliv = deliv.sort_values(by=['date_receipt'])

In [63]:
# Посмотрим на датасет
deliv.head(10)

,date_receipt,warehouse_id,PlacementToId,date_orders,PlacementFromId,IsFromSupplier,OFFER_ID
591,2020-12-23T00:00:00,7,5f219248-fadd-11df-8a53-00262d126e8b,2020-12-18T00:00:00,d295458c-d7cf-11ea-ab3a-000c29a0142b,1,eddbc7df-0a09-11eb-ab3d-000c29a0142b#eddbc7e1-...
1164,2020-12-23T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-11T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...
1165,2020-12-23T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-16T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...
1166,2020-12-23T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-17T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...
1167,2020-12-23T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-21T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...
1168,2020-12-23T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-25T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...
1169,2020-12-23T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-26T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...
1170,2020-12-23T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-27T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...
1171,2020-12-23T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-12-04T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...
1172,2020-12-23T00:00:00,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-12-05T00:00:00,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...


In [64]:
# Из типа объект делаем даты
object_to_date(deliv, deliv.date_receipt, 'date_receipt')
object_to_date(deliv, deliv.date_orders, 'date_orders')

In [65]:
# Создадим предиктор со сроками поставок
deliv['difference'] = deliv.date_receipt - deliv.date_orders

# Посмотрим результат
deliv.head(10)

,date_receipt,warehouse_id,PlacementToId,date_orders,PlacementFromId,IsFromSupplier,OFFER_ID,difference
591,2020-12-23,7,5f219248-fadd-11df-8a53-00262d126e8b,2020-12-18,d295458c-d7cf-11ea-ab3a-000c29a0142b,1,eddbc7df-0a09-11eb-ab3d-000c29a0142b#eddbc7e1-...,5 days
1164,2020-12-23,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-11,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...,42 days
1165,2020-12-23,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-16,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...,37 days
1166,2020-12-23,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-17,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...,36 days
1167,2020-12-23,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-21,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...,32 days
1168,2020-12-23,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-25,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...,28 days
1169,2020-12-23,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-26,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...,27 days
1170,2020-12-23,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-27,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...,26 days
1171,2020-12-23,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-12-04,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...,19 days
1172,2020-12-23,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-12-05,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...,18 days


In [66]:
# Выберем только количество 
# дней в сроках поставок
lst = []
for item in deliv.difference:
    lst.append(item.days)
deliv['delivery'] = lst

# Посмотрим информацию по датасету
deliv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 870 entries, 591 to 1460
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   date_receipt     870 non-null    datetime64[ns] 
 1   warehouse_id     870 non-null    int64          
 2   PlacementToId    870 non-null    object         
 3   date_orders      870 non-null    datetime64[ns] 
 4   PlacementFromId  870 non-null    object         
 5   IsFromSupplier   870 non-null    int64          
 6   OFFER_ID         870 non-null    object         
 7   difference       870 non-null    timedelta64[ns]
 8   delivery         870 non-null    int64          
dtypes: datetime64[ns](2), int64(3), object(3), timedelta64[ns](1)
memory usage: 68.0+ KB


In [126]:
# Результат предыдущих операций
deliv.head()

,date_receipt,warehouse_id,PlacementToId,date_orders,PlacementFromId,IsFromSupplier,OFFER_ID,difference,delivery
591,2020-12-23,7,5f219248-fadd-11df-8a53-00262d126e8b,2020-12-18,d295458c-d7cf-11ea-ab3a-000c29a0142b,1,eddbc7df-0a09-11eb-ab3d-000c29a0142b#eddbc7e1-...,5 days,5
1164,2020-12-23,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-11,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...,42 days,42
1165,2020-12-23,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-16,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...,37 days,37
1166,2020-12-23,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-17,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...,36 days,36
1167,2020-12-23,15,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-21,5464cd6c-b080-11e2-85e4-00155d006702,1,33b1e847-ca98-11e7-ab61-000c2945f18b#33b1e849-...,32 days,32


In [129]:
# Создадим таблицу со с следующими предикторами
index_cols = ['PlacementFromId', 'PlacementToId', 'warehouse_id', 'p10Dates', 'highDates', 'lowDates']

grid = []

for fabricator in deliv['PlacementFromId'].unique():
    cur_warehouse = deliv.loc[deliv['PlacementFromId'] == fabricator]
    
    deliv_min = cur_warehouse.groupby(['PlacementFromId', 'PlacementToId', 'warehouse_id']).agg(
        {'delivery': percentile(50)}).reset_index()
    deliv_max = cur_warehouse.groupby(['PlacementFromId', 'PlacementToId', 'warehouse_id']).agg(
        {'delivery': percentile(90)}).reset_index()
    deliv_mean = cur_warehouse.groupby(['PlacementFromId', 'PlacementToId', 'warehouse_id']).agg(
        {'delivery': np.mean}).reset_index()

    deliv_max['p10Dates'] = deliv_max.delivery
    deliv_min['highDates'] = deliv_min.delivery
    deliv_mean['lowDates'] = deliv_mean.delivery
    deliv_max.drop('delivery', axis=1, inplace=True)
    deliv_min.drop('delivery', axis=1, inplace=True)
    deliv_mean.drop('delivery', axis=1, inplace=True)
    
    statistics = pd.merge(
        deliv_max, deliv_min,
        on=['PlacementFromId', 'PlacementToId', 'warehouse_id'],
        how='left'
    )
    
    statistics = pd.merge(
        statistics, deliv_mean,
        on=['PlacementFromId', 'PlacementToId', 'warehouse_id'],
        how='left'
    )
    
    grid.append(statistics)
    
stat = pd.DataFrame(np.vstack(grid), columns = index_cols)
stat

,PlacementFromId,PlacementToId,warehouse_id,p10Dates,highDates,lowDates
0,d295458c-d7cf-11ea-ab3a-000c29a0142b,5f219248-fadd-11df-8a53-00262d126e8b,7,5,5,5
1,5464cd6c-b080-11e2-85e4-00155d006702,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,45,83.2,48,57.3333
2,5464cd6c-b080-11e2-85e4-00155d006702,5f219248-fadd-11df-8a53-00262d126e8b,7,86,86,75
3,5464cd6c-b080-11e2-85e4-00155d006702,f6548095-c106-11e0-8baa-00262d126e8b,15,34,21,21.9428
4,4e749769-f4bd-11e9-a2dc-000c29675a27,5f219248-fadd-11df-8a53-00262d126e8b,7,41.6,40,37.6667
...,...,...,...,...,...,...
74,c6014a86-283c-11e8-ab63-000c2945f18b,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,45,25.6,24,24
75,9afa19ce-7902-11e8-a2c4-000c29dcdf3d,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,45,26,26,26
76,bb25f8cf-ab02-11e2-911e-f46d04736e4a,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,45,59.1,47.5,47.5
77,085f3915-84c3-11e2-be7c-f46d04736e4a,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,45,77,77,77


In [123]:
grid

""


In [114]:
cur_warehouse = deliv.loc[deliv['PlacementFromId'] == '1f688887-b7fe-11e8-a2c1-00505607eae9']
cur_warehouse

,date_receipt,warehouse_id,PlacementToId,date_orders,PlacementFromId,IsFromSupplier,OFFER_ID,difference,delivery
1437,2020-12-23,7,5f219248-fadd-11df-8a53-00262d126e8b,2020-11-10,1f688887-b7fe-11e8-a2c1-00505607eae9,1,b2534b05-c6d1-11e8-a2c1-00505607eae9#d6929e5c-...,43 days,43
1432,2020-12-23,7,5f219248-fadd-11df-8a53-00262d126e8b,2020-10-18,1f688887-b7fe-11e8-a2c1-00505607eae9,1,c7d2abbe-f294-11ea-ab3c-000c29a0142b#c7d2abc0-...,66 days,66


In [115]:
deliv_min = cur_warehouse.groupby(['PlacementFromId', 'PlacementToId', 'warehouse_id']).agg(
    {'delivery': percentile(50)}).reset_index()
deliv_max = cur_warehouse.groupby(['PlacementFromId', 'PlacementToId', 'warehouse_id']).agg(
    {'delivery': percentile(90)}).reset_index()
deliv_mean = cur_warehouse.groupby(['PlacementFromId', 'PlacementToId', 'warehouse_id']).agg(
    {'delivery': np.mean}).reset_index()

In [116]:
deliv_max

,PlacementFromId,PlacementToId,warehouse_id,delivery
0,1f688887-b7fe-11e8-a2c1-00505607eae9,5f219248-fadd-11df-8a53-00262d126e8b,7,63.7


In [117]:
deliv_min

,PlacementFromId,PlacementToId,warehouse_id,delivery
0,1f688887-b7fe-11e8-a2c1-00505607eae9,5f219248-fadd-11df-8a53-00262d126e8b,7,54.5


In [118]:
deliv_mean

,PlacementFromId,PlacementToId,warehouse_id,delivery
0,1f688887-b7fe-11e8-a2c1-00505607eae9,5f219248-fadd-11df-8a53-00262d126e8b,7,54.5


In [119]:
deliv_max['p10Dates'] = deliv_max.delivery
deliv_min['highDates'] = deliv_min.delivery
deliv_mean['lowDates'] = deliv_mean.delivery
deliv_max.drop('delivery', axis=1, inplace=True)
deliv_min.drop('delivery', axis=1, inplace=True)
deliv_mean.drop('delivery', axis=1, inplace=True)

In [121]:
statistics = pd.merge(
        deliv_max, deliv_min,
        on=['PlacementFromId', 'PlacementToId', 'warehouse_id'],
        how='left'
    )

TypeError: Can only merge Series or DataFrame objects, a <class 'list'> was passed

In [111]:
statistics

,PlacementFromId,PlacementToId,warehouse_id,p10Dates,highDates
0,1f688887-b7fe-11e8-a2c1-00505607eae9,5f219248-fadd-11df-8a53-00262d126e8b,7,63.7,54.5


In [112]:
statistics = pd.merge(
        statistics, deliv_mean,
        on=['PlacementFromId', 'PlacementToId', 'warehouse_id'],
        how='left'
    )

In [113]:
statistics

,PlacementFromId,PlacementToId,warehouse_id_x,p10Dates,highDates,warehouse_id_y,lowDates
0,1f688887-b7fe-11e8-a2c1-00505607eae9,5f219248-fadd-11df-8a53-00262d126e8b,7,63.7,54.5,7,54.5


In [ ]:
deliv_min = cur_warehouse.groupby(['PlacementFromId', 'PlacementToId']).agg(
    {'delivery': percentile(50)}).reset_index()
deliv_max = cur_warehouse.groupby(['PlacementFromId', 'PlacementToId', 'IsFromSupplier']).agg(
    {'delivery': percentile(90)}).reset_index()

In [152]:
deliv_max['DaysTo'] = deliv_max.delivery
deliv_min['DaysFrom'] = deliv_min.delivery
deliv_max.drop('delivery', axis=1, inplace=True)
deliv_min.drop('delivery', axis=1, inplace=True)

In [153]:
statistics = pd.merge(
    deliv_max, deliv_min,
    on=['PlacementFromId', 'PlacementToId', 'IsFromSupplier'],
    how='left'
)
statistics

,PlacementFromId,PlacementToId,IsFromSupplier,DaysTo,DaysFrom
0,00000000-0000-0000-0000-000000000000,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,1,63.0,63.0
1,00000000-0000-0000-0000-000000000000,5f219248-fadd-11df-8a53-00262d126e8b,1,50.0,50.0
2,00bae129-1faf-11e9-a2c8-00505607eae9,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,1,67.3,37.0
3,00bae129-1faf-11e9-a2c8-00505607eae9,385b9ff4-cccf-11ea-ab39-000c29a0142b,1,53.8,51.0
4,00bae129-1faf-11e9-a2c8-00505607eae9,4bc4d898-ec0e-11e9-a2dc-000c29675a27,1,59.8,39.0
...,...,...,...,...,...
1511,fed611d8-cb93-11e4-b853-f46d04736e4a,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,1,62.3,54.0
1512,fed611d8-cb93-11e4-b853-f46d04736e4a,5f219248-fadd-11df-8a53-00262d126e8b,1,63.0,48.0
1513,fed611d8-cb93-11e4-b853-f46d04736e4a,a2f89531-75de-11e8-a2c3-000c29dcdf3d,1,67.2,56.0
1514,fed611d8-cb93-11e4-b853-f46d04736e4a,f6548095-c106-11e0-8baa-00262d126e8b,1,91.0,65.0


In [154]:
statistics.to_json('statistics.json', orient="split")
statistics.to_excel('statistics.xlsx', index=False)  

### To be continued...

In [155]:
deliv

,date_receipt,PlacementToId,date_orders,PlacementFromId,IsFromSupplier,difference,delivery
88752,2020-09-19,03f70e82-3e4e-11e8-a2bb-000c29dcdf3d,2020-08-13,53898bbe-d2b7-11e8-a2c4-00505607eae9,1,37 days,37
88875,2020-09-19,5f219248-fadd-11df-8a53-00262d126e8b,2020-08-03,48452082-00c4-11e8-ab61-000c2945f18b,1,47 days,47
88876,2020-09-19,5f219248-fadd-11df-8a53-00262d126e8b,2020-07-03,48452082-00c4-11e8-ab61-000c2945f18b,1,78 days,78
88877,2020-09-19,f6548095-c106-11e0-8baa-00262d126e8b,2020-07-11,4038bae0-e7bc-11e6-80d6-00155d000e02,1,70 days,70
88878,2020-09-19,f6548095-c106-11e0-8baa-00262d126e8b,2020-07-06,f0287a2c-d6a6-11e4-891c-f46d04736e4a,1,75 days,75
...,...,...,...,...,...,...,...
159471,2020-12-17,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-18,d86359ec-e37f-11ea-ab3c-000c29a0142b,1,29 days,29
159472,2020-12-17,f6548095-c106-11e0-8baa-00262d126e8b,2020-11-29,d86359ec-e37f-11ea-ab3c-000c29a0142b,1,18 days,18
159473,2020-12-17,f6548095-c106-11e0-8baa-00262d126e8b,2020-12-10,d86359ec-e37f-11ea-ab3c-000c29a0142b,1,7 days,7
159464,2020-12-17,4bc4d898-ec0e-11e9-a2dc-000c29675a27,2020-10-15,9850e147-7c17-11e7-ab50-000c2945f18b,1,63 days,63


In [157]:
deliv.to_csv('delivery_time.csv', index=False)  